In [1]:
# SO using the SimOpt library

In [1]:
import simopt
from simopt import base, models, solvers, experiment_base
from mrg32k3a.mrg32k3a import MRG32k3a

## M/M/1 Queue

Here we run a single replication of an M/M/1 queue and report several outputs of interest. 

In [2]:
# Set up model
fixed_factors = {"lambda": 3.0, "mu": 8.0}
mymodel = models.mm1queue.MM1Queue(fixed_factors)
# Generate random numbers
# Here mymodel.n_rngs is 2 - the number of random number streams we use - one for service times, one for inter arrival times
rng_list = [MRG32k3a(s_ss_sss_index=[0, ss, 0]) for ss in range(mymodel.n_rngs)]


In [9]:
# Report data collection details
print('Model Parameters')
print('----------------')
print('The arrival rate is: ' + str(fixed_factors['lambda']))
print('The service rate is: ' + str(fixed_factors['mu']))

print('\nData collection')
print('----------------')
print('We allow a warm up of ' + str(mymodel.specifications['warmup']['default']) + ' people before calculating statistics based on ' + str(mymodel.specifications['people']['default']) + ' people.')

# Run a single replication of the model.
responses, gradients = mymodel.replicate(rng_list)
print('\nOutputs')
print('----------------')
print("For a single replication:")
print("Responses:")
for key, value in responses.items():
    print(f"\t {key} is {round(value,2)}.")

Model Parameters
----------------
The arrival rate is: 3.0
The service rate is: 8.0

Data collection
----------------
We allow a warm up of 20 people before calculating statistics based on 50 people.

Outputs
----------------
For a single replication:
Responses:
	 avg_sojourn_time is 0.32.
	 avg_waiting_time is 0.18.
	 frac_cust_wait is 0.54.


## Minimising the mean sojourn time in the M/M/1 queue

Here we introduce an objective function and solution space for a problem involving the M/M/1 queue. 

In [11]:
myproblem = models.mm1queue.MM1MinMeanSojournTime()
print ('\n The decision variable is the service rate: ' + str(myproblem.model_decision_factors))


 The decision variable is the service rate: {'mu'}


There is a cost $c$ associated with increasing the service rate $\mu$. 

In [12]:
print(' c: ' + str(myproblem.specifications['cost']['default']) + ' is the cost per unit increase of service rate.')

 c: 0.1 is the cost per unit increase of service rate.


The objective $f(\mu; \xi, c) = y(\mu; \xi) + c**2$ where $y(\cdot)$ is the mean sojourn time and we wish to minimise this objective function. 

In [16]:
print('Solution space')
print('----------------')
print('The solution space is ' + str(myproblem.variable_type) + ' with lower bound ' + str(myproblem.lower_bounds[0]) + ' and upper bound ' + str(myproblem.upper_bounds[0]))

print('\nModel Parameters')
print('----------------')
print('lambda: ' + str(myproblem.model.specifications['lambda']['default']) + ' is the arrival rate.')

print('\nData collection')
print('----------------')
print('We allow a warm up of ' + str(mymodel.specifications['warmup']['default']) + ' people before calculating statistics based on ' + str(mymodel.specifications['people']['default']) + ' people.')

Solution space
----------------
The solution space is continuous with lower bound 0 and upper bound inf

Model Parameters
----------------
lambda: 1.5 is the arrival rate.

Data collection
----------------
We allow a warm up of 20 people before calculating statistics based on 50 people.


We now run 10 simulation replications for a given solution and report the objective function values. 

In [17]:
x = (8,)
mysolution = base.Solution(x, myproblem)

# Create and attach rngs to solution
rng_list = [MRG32k3a(s_ss_sss_index=[0, ss, 0]) for ss in range(myproblem.model.n_rngs)]
mysolution.attach_rngs(rng_list, copy=False)

# Simulate a fixed number of replications (n_reps) at the solution x.
n_reps = 10
myproblem.simulate(mysolution, m=n_reps)

# Print results to console.
print(f"Ran {n_reps} replications of the {myproblem.name} problem at solution x = {x}.\n")
print(f"The mean objective estimate was {round(mysolution.objectives_mean[0], 4)} with standard error {round(mysolution.objectives_stderr[0], 4)}.")
print("The individual observations of the objective were:")
for idx in range(n_reps):
    print(f"\t {round(mysolution.objectives[idx][0], 4)}")

Ran 10 replications of the MM1-1 problem at solution x = (8,).

The mean objective estimate was 6.5576 with standard error 0.0051.
The individual observations of the objective were:
	 6.5544
	 6.5771
	 6.5411
	 6.5453
	 6.5431
	 6.535
	 6.5687
	 6.5706
	 6.5798
	 6.5604


## Testing the Random Search solver

Here we use the Random search solver to look for an optimal solution to this problem. We report the performance of this solver